<a href="https://colab.research.google.com/github/sarikasea/Streamlit-Interactive-Dashboards/blob/main/Streamlit_Upload_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit ngrok plotly pandas pyngrok


In [2]:
!pip install wbdata

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

import wbdata

In [4]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import wbdata
import datetime

st.set_page_config(layout="wide")
st.title("Combine Uploaded/Random Data with World Bank Data")

option = st.radio("Choose your data source:", [
    "Random",
    "Upload CSV",
    "World Bank (wbdata)"
])

df = None  # will hold your main data

if option == "Random":
    df = pd.DataFrame(np.random.randn(50, 3), columns=["A", "B", "C"])

elif option == "Upload CSV":
    uploaded = st.file_uploader("Choose a CSV", type="csv")
    if uploaded:
        df = pd.read_csv(uploaded)
    else:
        st.stop()

elif option == "World Bank (wbdata)":
    countries = st.multiselect(
        "Select countries (ISO codes):",
        ["USA", "CHN", "IND", "BRA", "CAN", "GBR", "FRA", "DEU", "JPN", "AUS"],
        default=["USA", "CHN"]
    )
    indicators_dict = {
        "GDP (current US$)": "NY.GDP.MKTP.CD",
        "Population": "SP.POP.TOTL",
        "Life expectancy": "SP.DYN.LE00.IN",
    }
    indicator_name = st.selectbox("Select indicator:", list(indicators_dict.keys()))
    indicator_id = indicators_dict[indicator_name]
    start_year = st.slider("Start year", 2000, 2022, 2015)
    end_year = st.slider("End year", 2000, 2022, 2022)
    if st.button("Load World Bank Data"):
        with st.spinner("Fetching data..."):
            dates = (datetime.datetime(start_year, 1, 1), datetime.datetime(end_year, 12, 31))
            df = wbdata.get_dataframe(
                {indicator_id: indicator_name},
                country=countries,
                data_date=dates,
                convert_date=True
            )
            df = df.reset_index()
        st.success(f"Loaded {len(df)} rows.")

# Show and analyze data if present
if df is not None:
    st.dataframe(df)

    st.subheader("Summary Stats")
    st.write(df.describe())

    # Example: plot if at least two numeric columns are available
    numeric_cols = df.select_dtypes(include='number').columns
    if len(numeric_cols) > 1:
        x_axis = st.selectbox("X axis", numeric_cols)
        y_axis = st.selectbox("Y axis", numeric_cols, index=1)
        fig = px.scatter(df, x=x_axis, y=y_axis)
        st.plotly_chart(fig)
    else:
        st.write("Not enough numeric columns to plot.")
else:
    st.info("Choose data source to begin.")

# --- Example: combine with another source (advanced) ---
# Representative if you want to merge/join data sources
# The user can select/merge a second source in a similar way,
# then use pd.concat([df1, df2]) for vertical stacking



2025-07-29 04:59:06.409 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 04:59:06.411 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 04:59:06.797 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-29 04:59:06.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 04:59:06.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 04:59:06.807 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-29 04:59:06.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [5]:
import os
from pyngrok import ngrok
import threading
import time

from google.colab import userdata

# Set ngrok authtoken (must be in Colab Secrets)
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
if ngrok_authtoken is None:
    raise ValueError("NGROK_AUTHTOKEN not found in Colab Secrets. Please add it.")
os.environ["NGROK_AUTHTOKEN"] = ngrok_authtoken

# Threaded Streamlit runner
def run():
    os.system("streamlit run streamlit_app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false --browser.gatherUsageStats false")

thread = threading.Thread(target=run)
thread.start()

time.sleep(5)  # Wait for Streamlit to launch

# Start ngrok tunnel and print URL
public_url = ngrok.connect(8501).public_url
print(f"Streamlit app public URL: {public_url}")


Streamlit app public URL: https://f88140eddc00.ngrok-free.app
